In [ ]:
%matplotlib inline
#!pip install -r requirements.txt
import numpy as np
import pandas as pd
import us
import re
import geocoder
import regex
import re
import matplotlib.pyplot as plt


In [285]:
us_state_names = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho",
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
    "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina",
    "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
    "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah",
    "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]
us_states_abbr = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", 
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", 
    "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", 
    "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", 
    "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", 
    "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", 
    "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", 
    "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", 
    "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", 
    "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", 
    "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", 
    "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", 
    "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT", 
    "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", 
    "Wisconsin": "WI", "Wyoming": "WY"
}
us_state_correction = {
        "au large de la californie": "California",
        "Pacifique":"California",
        "au large de la Californie":"California",
        "nouveau mexique": "New Mexico",
        "chicago": "Illinois",
        "los angeles": "California",
        "swisher districts": "Texas",
        "rhode isl. ": "Rhode Island",
        "near los angeles": "California",
        "atlantic terminal station": "New York",
        "north california": "None",
        "los angeles": "California",
        "jerauld": "South Dakota",
        "jefferson davis": "Mississippi",
        "nassau-florida": "Florida",
        "wilkes": "North Carolina",
        "noble in richland": "Illinois",
        "marion-in": "Indiana",
        "central michigan ": "Michigan",
        "south california": "California",
        "south-western louisiana": "Louisiana",
        "north california ": "California",
        "northern washington ": "Washington",
        "central-northern usa": "None",
        "montana and idaho s": "None",
        "hollywood": "California",
        "eastern washington ": "Washington",
        "eastern new mexico": "New Mexico",
        "district of columbia":"Washington, D.C.",
        "Nouveau Mexique" :"New Mexico",
        "Floride" : "Florida"
    }

In [ ]:
data = pd.read_csv('location.csv',names=["Location"])
data

In [ ]:
data

In [ ]:
def state_boundary(x):
    if re.match(".*south|north|east|west.*", x.lower()):
        print("..",x)
    if re.match("(.*);(.*)", x.lower()): 
        multistate = x.split(";")
        multistate_str = ""
        for ms in multistate:
            print("..",ms,us.states.lookup(ms.strip()))
            if us.states.lookup(ms.strip()):
                state = us.states.lookup(ms).__str__()
            else:
                state =""
            multistate_str.join(state)
            print("multistate",multistate_str)
        x=multistate_str
    return x

data['Location'].map(lambda x : state_boundary(x))


In [ ]:
txt = "Florida; Georgia; North Carolina; South Carol"
re.match(".*south|north|east|west.*", txt.lower())

In [ ]:
def segregate_state(loc,df):
    indx = data[data['Location'] == loc].index # Lets collect the index of this row to use it later in this section
    
    if not loc is np.nan:
        loc_list = loc.split(';') # split all the location which contains ',' to segregate state(s)
        if len(loc_list) == 0:
            # This line is if the location contains valid state. no need to continue further.
            if not us.states.lookup(loc.strip()):
                state = us.states.lookup(loc).__str__()
                df.loc[indx,"Location"] = state
                print("Correct State single",state)
            return df

        # Check if the row needs to be duplicated with more than one province
        provinces = 0
        state = ""
        for ll in loc_list:
            # call func
            #segregate_state(indx,ll,data)
   
            matched = re.match('.*\((.*)\)',ll) # Match the brackets with regular expression
            if 'province' in ll.lower() or ll.capitalize().strip() in us_state_names or ll.strip().lower() in us_state_correction.keys() or matched:
                provinces+=1

                if matched:
                    state =matched[1]
                else:
                    print("stores as ,",ll)
                    state = ll
                    state = re.sub(r"state|provinces|province", "", state).lower()
                    state = re.sub(r"\)|\(", "", state).lower()
                    print("Prov",state)
                if state in us_state_correction:
                    print("incorrect",state)
                    state = us_state_correction[state] # Get the correct state name from the dictionary
                    print("correct",state)
                if us.states.lookup(state.strip()):
                    state = us.states.lookup(state).__str__()
                if provinces >1:
                    # We will duplicate the row with new state name
                    new_df=df.iloc[indx].copy()
                    new_df['Location_new'] = state.capitalize().strip()
                    df=df._append([new_df],ignore_index=True)
                else:                    
                    state =state.capitalize().strip()
                    df.loc[indx,"Location_new"] = state
    return df

In [ ]:
data_new
#.explode('Location_new').reset_index(drop=True)
    

In [ ]:
data = pd.read_csv('location.csv',names=["Location"])
data


In [ ]:
pattern = '|'.join(['provinces', 'province'])

data['Location'] = data['Location'].str.replace(pattern, '', regex=True)
data['Location_new'] = data['Location'].apply(lambda x: ",".join(re.findall(r'\((.*?)\)', x)))

#data_new = data.explode('Location_new').reset_index(drop=True)
#data['Location_new'] = data['Location'].apply(lambda x: "; ".join(re.findall(r'\((.*?)\)', x)))
# Explode the extracted list into multiple rows
data.explode('Location_new').reset_index(drop=True)

In [ ]:
def extract_state(x):
    if "(" in x:
        #x = x.replace(";",",")
        return re.findall(r'\((.*?)\)', x)
    if "," in x:
        return x.split(",")
    return [x]

# This line is if the location contains valid state. no need to continue further.
def segregate_state(x):
    x = re.sub(r'\s+s$', '', x)
    x = x.strip()
    x = re.sub(r'and |near', '', x) 
    x = re.sub(r'^.*?,\s*', '', x)
    if x in us_state_correction.keys():
        x = us_state_correction[x]
    x_state = us.states.lookup(x).__str__()
    if not x_state == 'None':
        return x_state.capitalize().strip()
    else:
        # Find words that contain the substring
        x_state = [st for st in us_states_abbr.keys() if  st.lower() in x]
        if len(x_state) > 0:
            return x_state
        else:
            return x
        
    

In [ ]:
def segregate_state(x):
    x = re.sub(r'\s+s$', '', x)
    x = x.strip()
    x = re.sub(r'and |near', '', x) 
    x = re.sub(r'^.*?,\s*', '', x)
    if x in us_state_correction.keys():
        x = us_state_correction[x]
    x_state = us.states.lookup(x).__str__()
    if not x_state == 'None':
        return x_state.capitalize().strip()
    else:
        # Find words that contain the substring
        x_state = [st for st in us_states_abbr.keys() if  st.lower() in x]
        if len(x_state) > 0:
            return x_state
        else:
            return x
segregate_state("")

In [353]:
# List of words
word_list = ["apple", "banana", "cherry", "grape", "pineapple"]
# Part of word to check
substring = " bell s"

# Find words that contain the substring
matching_words = [word for word in word_list if  word.lower() in substring]
print("Words containing the substring:", matching_words)

Words containing the substring: []


In [358]:
import pandas as pd
import re
data = pd.read_csv('location.csv',names=["Location"])
# Create the DataFrame
#data = pd.DataFrame({'Location': ["bullock; montgomery districts (alabama province); colquitt; grady; mitchell; thomas; tift districts (georgia province); washington district (floride province); lonoke; prairie; saline districts (arkansas province)"]})
pattern = '|'.join(['province', 'provinces','state','State','nan'])
# Extract text inside parent heses and split by ';'
data['Location'] = data['Location'].str.replace(pattern, '', regex=True)
data['Location'] =data['Location'].replace(";",",",regex=True)
#data['Extracted'] = data['Location'].apply(lambda x: re.findall(r'\((.*?)\)', x))
data['Extracted'] = data['Location'].apply(extract_state)

# Create a new DataFrame by stacking the split elements into new rows
#exploded_df = pd.concat([pd.DataFrame({'Location': data.loc[idx, 'Location'], 'Extracted': item}, index=[idx]) 
#                         for idx, items in data['Extracted'].items() 
#                         for item in items]).reset_index(drop=True)
# Explode the extracted list into multiple rows
df_exploded = data.explode('Extracted').reset_index(drop=True)
df_exploded['Extracted'] = df_exploded['Extracted'].apply(segregate_state)

df_exploded.dropna(axis=0,inplace=True)
df_exploded.reset_index(drop=True)


,Location,Extracted
0,"Crittenden, Daviess, Webster districts (Kentuc...",Kentucky
1,Pacifique (au large de la Californie),California
2,"Bullock, Montgomery districts (Alabama ), Col...",Alabama
3,"Bullock, Montgomery districts (Alabama ), Col...",Georgia
4,"Bullock, Montgomery districts (Alabama ), Col...",Florida
...,...,...
3239,"Florida, Georgia, North Carolina, South Carol...",South carolina
3240,"Florida, Georgia, North Carolina, South Carol...",Tennessee
3241,"Florida, Georgia, North Carolina, South Carol...",Kentucly Virginia
3242,"Hillsborough, Pinellas, Sarasota Volusia, Sai...",Florida


In [377]:
uniq = df_exploded[df_exploded['Extracted'].isin(us_state_names)]['Extracted'].unique()
difference2 = list(set(us_states_abbr.keys())-set(uniq))
for diff in difference2:
    print(f"{diff}")

New York
North Dakota
New Jersey
New Hampshire
South Carolina
New Mexico
North Carolina
West Virginia
South Dakota
Rhode Island


In [379]:
if "Tennessee" in df_exploded['Extracted'].to_list():
    print("correct ")

correct 


In [380]:

difference2 = list(set(df_exploded['Extracted'].to_list()) - set(us_states_abbr.keys()))


us_states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho",
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi",
    "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey",
    "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma",
    "Oregon", "Pennsylvania", "South Carolina", "South Dakota", "Tennessee", "Texas",
    "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]




for diff in difference2:
    print(f"original {diff}")
    


TypeError: unhashable type: 'list'

In [339]:
x = "Rhode Isl."
print(type(x))
x_state = us.states.lookup(x).__str__()
#.capitalize().strip()
print(x_state)
if not x_state == 'None':
    print(x_state)
else:
    print("as is ",x)

<class 'str'>
None
as is  Rhode Isl.


Best match for 'northern Texas': Texas with score 90


In [ ]:
#df_exploded['Extracted']=

#df_exploded

In [ ]:
import re
import pandas as pd

data = pd.read_csv('location.csv',names=["Location"])
# Function to extract the text inside parentheses
def extract_state(location):
    return re.findall(r'\((.*?)\)', location)  # Ensures the output is always a list

# Apply cleaning and extraction
pattern = '|'.join(['province', 'provinces', 'state'])
data['Location'] = data['Location'].str.replace(pattern, '', regex=True)
data['Extracted'] = data['Location'].apply(extract_state)

# Explode the extracted list into multiple rows
df_exploded = data.explode('Extracted').reset_index(drop=True)

df_exploded


In [ ]:
# Fix the following :
# We have some states in brackets '()'
# the state name is attached with text 'province' 
# The state name is attached with text 'state'
# Some location column has more than one state combined in one row. We will duplicate those rows and add rows for each state in seperate rows
# We will also fix letter cases.The capitalize() method returns a string where the first character is upper case, and the rest is lower case.

#data_usa = data[data.Country == 'United States of America'].reset_index()
locations=list(data['Location'])

cnt = 0
for loc in locations:
    print("loc is ",loc)
    # call func
    data = segregate_state(loc,data)
    cnt+=1
    if cnt == 3:
        
        break

data["Location"] = data["Location"].str.capitalize()   
         
data

In [ ]:
data['Location']

In [ ]:
import pandas as pd

#df = pd.DataFrame({'A': ['LOCAL FOREIGN', 'TEST FOREIGN', 'ANOTHER HELLO', 'NOTHING']})
 
df = pd.DataFrame({'A': ["bullock; montgomery districts (alabama province); colquitt; grady; mitchell; thomas; tift districts (georgia province); washington district (floride province); lonoke; prairie; saline districts (arkansas province)"]})
pattern = '|'.join(['LOCAL', 'FOREIGN', 'HELLO'])
#print(df)
df['A'] = df['A'].str.replace(pattern, 'CORP', regex=True)
pattern = '|'.join(['province', 'provinces', '(',')'])
df['A'] = df['A'].str.replace(pattern, '', regex=True)
# Regex to extract only the words inside parentheses
txt = "bullock; montgomery districts (alabama province); colquitt; grady; mitchell; thomas; tift districts (georgia province); washington district (floride province); lonoke; prairie; saline districts (arkansas province)"
result = re.findall(r'\((.*?)\)', txt)

# Join results into a single string if desired
output = "; ".join(result)
#.replace(pattern,'',regex=True)
df['A'].values
output
df['Extracted'] = df['A'].apply(lambda x: "; ".join(re.findall(r'\((.*?)\)', x)))
df